In [69]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

df = pd.read_csv("billboard100_data.csv")
df_cluster = pd.read_csv("clustered_data.csv")


In [38]:
df['title'] = df['title'].map(str.lower) # converting the entries to lower
df['author'] = df['author'].map(str.lower) # converting the entries to lower
df

,title,author
0,flowers,miley cyrus
1,kill bill,sza
2,last night,morgan wallen
3,unholy,sam smith & kim petras
4,creepin',"metro boomin, the weeknd & 21 savage"
...,...,...
95,yandel 150,yandel & feid
96,heart to heart,mac demarco
97,next thing you know,jordan davis
98,never gonna not dance again,p!nk


In [39]:
#from fuzzywuzzy import process
#def check_mistakes(song_name, reference_database, title_column, artist_column):
#    best_match, score = process.extractOne(song_name, reference_database[title_column].values)
#    if score >= 70:
#        artist = reference_database[reference_database[title_column] == best_match][artist_column][0]
#        answer = input("f\Did you mean {best_match} by {artist}? Y/N: ")
#        #answer = input(“”).lower().title()
#        if answer == "Y":
#            return best_match
#        if answer == "N":
#            pass
#    else:
#        print("error")

In [40]:
x = input('Enter a song you like: ')
x_artist = df.loc[df['title'].map(str.lower)  == x.lower(), 'author'] # artist corresponding to song x
if x.lower() in df.values:   # converts the entered x into all lower case. When x is in the billboard100
        rec_song = df['title'].sample().to_string(index=False) # recommendation song. I just removed the indexes as they are not required as the output  
        rec_artist = df.loc[df['title'] == rec_song, 'author'].to_string(index=False) # artist of the recommended song
        print("You might like this song: " + rec_song.title() + " by " + rec_artist.title())  # .title() makes the first alphabet in every word to upper   
else: # when the x is not in billboard100
    results = sp.search(q="Lose yourself",limit=3,market="GB")
results
        

Enter a song you like: Hotel California


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Lose+yourself&type=track&market=GB&offset=0&limit=3',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR'},
       'href': 'https://api.spotify.com/v1/artists/7dGJo4pcD2V6oG8kP0tJRR',
       'id': '7dGJo4pcD2V6oG8kP0tJRR',
       'name': 'Eminem',
       'type': 'artist',
       'uri': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/1rfORa9iYmocEsnnZGMVC4'},
     'href': 'https://api.spotify.com/v1/albums/1rfORa9iYmocEsnnZGMVC4',
     'id': '1rfORa9iYmocEsnnZGMVC4',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273b6ef2ebd34efb08cb76f6eec',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02b6ef2ebd34efb08cb76f6eec',
       'width': 300},
      {'height': 64,
       'url': 'https://i.

In [41]:
x = input('Enter a song you like: ')
results = sp.search(q=x,limit=3,market="GB")
results

Enter a song you like: Hotel California


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Hotel+California&type=track&market=GB&offset=0&limit=3',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0ECwFtbIWEVNwjlrfc6xoL'},
       'href': 'https://api.spotify.com/v1/artists/0ECwFtbIWEVNwjlrfc6xoL',
       'id': '0ECwFtbIWEVNwjlrfc6xoL',
       'name': 'Eagles',
       'type': 'artist',
       'uri': 'spotify:artist:0ECwFtbIWEVNwjlrfc6xoL'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2widuo17g5CEC66IbzveRu'},
     'href': 'https://api.spotify.com/v1/albums/2widuo17g5CEC66IbzveRu',
     'id': '2widuo17g5CEC66IbzveRu',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b2734637341b9f507521afa9a778',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e024637341b9f507521afa9a778',
       'width': 300},
      {'height': 64,
       'url': 'https://

In [42]:
# finding the id of the inputed song
track_id=track_id=results["tracks"]["items"][0]["id"]
track_id

'40riOy7x9W7GXjyGp4pjAv'

In [43]:
# Just want to play the inputed song
from IPython.display import IFrame

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [62]:
# Extract the audio feature of the inputed song
x_af = sp.audio_features(track_id) 
X_af = pd.DataFrame(x_af)
X_af_num = X_af.drop(['key', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)
# dropping some unwanted columns
X_af_num

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.579,0.508,-9.484,0.027,0.00574,0.000494,0.0575,0.609,147.125


In [71]:
# Scale the inputed song to the scale we used in function.ipynb file, i.e, the scale we used for our spotify playlist data set

df_spotify = pd.read_csv("spotify_data.csv")
X = df_spotify.drop(['id', 'duration_ms'], axis=1) # spotify data without id and duration
scaler = StandardScaler()
scaler_fitted = scaler.fit(X) # we need scaler_fitted for transforming the song which is to be inputed 
X_scaled = scaler_fitted.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns) # scaled the numerical feature data X

X_af_num_scaled = scaler_fitted.transform(X_af_num) # scaled according to the parameters from scaler_fitted (which you get from your data X)
X_af_num_scaled

array([[-0.61361891, -0.81493565, -0.64356528, -0.7668106 , -0.80060445,
        -0.40230121, -0.85660801,  0.35488222,  0.97267147]])

Predicting the cluster of the inputted song

In [75]:
# prediction
kmeans = KMeans(n_clusters=4, random_state=1234)
kmeans.fit(X_scaled_df)
song_cluster = kmeans.predict(X_af_num_scaled) # 

C:\Users\ajayk\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [76]:
song_cluster.


array([0])